In [42]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [ ]:
# Load environment variables
load_dotenv()

# Get database connection details from environment variables
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER') 
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')

# Create database connection string
db_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_string)

# Create connection and cursor
conn = psycopg2.connect(
| database=DB_NAME |
| user=DB_USER |
| password=DB_PASSWORD |
| host=DB_HOST |
| port=DB_PORT
)
cur = conn.cursor()

In [ ]:
# What are the top 5 brands by receipts scanned for most recent month?
top_brands_query = """ 
select
upper(split_part(replace(replace(replace(replace(brand_name |'''' |'') |'-' |'') |'%' |'') |'/' |'') | ' ' | 1)) clean_brand_name |
sum(brand_count) brand_sum
from
(select a.* | COUNT(*) brand_count from
(select
case
	when brand_code is not null and brand_code not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then brand_code
	when description is not null and description not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then description
	when user_flagged_description is not null and user_flagged_description not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then user_flagged_description
	else null
end as brand_name
-- barcode |description |brand_code |user_flagged_description |original_meta_brite_barcode |original_meta_brite_description
-- *
from receipts_items
where receipt_id in (
select id
from receipts
where date_trunc('month' | date_scanned) = (
	select date_trunc('month' | max_scan_date) - interval '1' month max_scan_month
		from 
			(select max(date_scanned) max_scan_date from receipts)
	)
)) a
where brand_name is not null
group by brand_name
order by 2 desc)
group by clean_brand_name 
order by brand_sum desc
limit 5

"""
# Execute the query
cur.execute(top_brands_query)
# Fetch the results
top_brands = cur.fetchall()
print("What are the top 5 brands by receipts scanned for most recent month?")
print("-"*50)
print("Top 5 brands by receipts scanned for most recent full month - Feb 2021:")
for brand | count in top_brands:
| print(f"{brand}: {count}")

What are the top 5 brands by receipts scanned for most recent month?
--------------------------------------------------
Top 5 brands by receipts scanned for most recent full month - Feb 2021:
FLIPBELT: 28
THINDUST: 27
MUELLER: 27
MILLER: 12
HEINZ: 10


In [ ]:
# How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
top_brands_query = """ 
select
upper(split_part(replace(replace(replace(replace(brand_name |'''' |'') |'-' |'') |'%' |'') |'/' |'') | ' ' | 1)) clean_brand_name |
sum(brand_count) brand_sum
from
(select a.* | COUNT(*) brand_count from
(select
case
	when brand_code is not null and brand_code not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then brand_code
	when description is not null and description not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then description
	when user_flagged_description is not null and user_flagged_description not in ('DELETED ITEM' |'ITEM NOT FOUND' |'BRAND') then user_flagged_description
	else null
end as brand_name
-- barcode |description |brand_code |user_flagged_description |original_meta_brite_barcode |original_meta_brite_description
-- *
from receipts_items
where receipt_id in (
select id
from receipts
where date_trunc('month' | date_scanned) = (
	select date_trunc('month' | max_scan_date) - interval '2' month max_scan_month
		from 
			(select max(date_scanned) max_scan_date from receipts)
	)
)) a
where brand_name is not null
group by brand_name
order by 2 desc)
group by clean_brand_name 
order by brand_sum desc
limit 5
"""
# Execute the query
cur.execute(top_brands_query)
# Fetch the results
previous_top_brands = cur.fetchall()
print("How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?")
print("-"*50)
print("Top 5 brands by receipts scanned for Jan 2021:")
for brand | count in previous_top_brands:
| print(f"{brand}: {count}")
print("-"*50)
print("Top 5 brands by receipts scanned for Feb 2021:")
for brand | count in top_brands:
| print(f"{brand}: {count}")

How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
--------------------------------------------------
Top 5 brands by receipts scanned for Jan 2021:
HYVEE: 304
BEN: 210
MILLER: 155
PC: 138
KLARBRUNN: 133
--------------------------------------------------
Top 5 brands by receipts scanned for Feb 2021:
FLIPBELT: 28
THINDUST: 27
MUELLER: 27
MILLER: 12
HEINZ: 10


In [ ]:
# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’/'Finished' or ‘Rejected’ | which is greater?
status_avg_spend_query = """ 
select rewards_receipt_status | ROUND(CAST(FLOAT8 (avg(total_spent)) AS NUMERIC) |2) as avg_total_spent
from
(select id |user_id |rewards_receipt_status |total_spent 
from receipts
where upper(rewards_receipt_status) in ('FINISHED' |'REJECTED')
group by id |user_id |rewards_receipt_status |total_spent)
group by rewards_receipt_status
order by avg_total_spent desc;
"""
# Execute the query
cur.execute(status_avg_spend_query)
# Fetch the results
status_avg_spend = cur.fetchall()
print("When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’/'Finished' or ‘Rejected’ | which is greater?")
print("-"*50)
for status | spend in status_avg_spend:
| print(f"{status}: ${spend}")


When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’/'Finished' or ‘Rejected’, which is greater?
--------------------------------------------------
FINISHED: $80.85
REJECTED: $23.33


In [ ]:
# When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’ | which is greater?
status_receipt_count_query = """ 
select rewards_receipt_status | cast(sum(purchased_item_count) as int) as total_receipts
from
(select id |user_id |rewards_receipt_status |purchased_item_count 
from receipts
where upper(rewards_receipt_status) in ('FINISHED' |'REJECTED')
group by id |user_id |rewards_receipt_status |purchased_item_count)
group by rewards_receipt_status
order by total_receipts desc;
"""
# Execute the query
cur.execute(status_receipt_count_query)
# Fetch the results
status_receipt_count = cur.fetchall()
print("When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’ | which is greater?")
print("-"*50)
for status | count in status_receipt_count:
| print(f"{status}: {count}")

When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
--------------------------------------------------
FINISHED: 8184
REJECTED: 173


In [ ]:
# Which brand has the most spend among users who were created within the past 6 months?
recent_users_top_brands_query = """ 
select count(*)
from users where created_date >= (NOW() - interval '6' month)
"""
# Execute the query
cur.execute(recent_users_top_brands_query)
# Fetch the results
recent_users_top_brands = cur.fetchall()
print("Which brand has the most spend among users who were created within the past 6 months?")
print("-"*50)
for count in recent_users_top_brands:
| print(f"Number of users created within the past 6 months: {count[0]}")

Which brand has the most spend among users who were created within the past 6 months?
--------------------------------------------------
Number of users created within the past 6 months: 0


In [ ]:
# Which brand has the most transactions among users who were created within the past 6 months?
recent_users_top_brands_query = """ 
select count(*)
from users where created_date >= (NOW() - interval '6' month)
"""
# Execute the query
cur.execute(recent_users_top_brands_query)
# Fetch the results
recent_users_top_brands = cur.fetchall()
print("Which brand has the most transactions among users who were created within the past 6 months?")
print("-"*50)
for count in recent_users_top_brands:
| print(f"Number of users created within the past 6 months: {count[0]}")

Which brand has the most transactions among users who were created within the past 6 months?
--------------------------------------------------
Number of users created within the past 6 months: 0
